In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
from loss_funcs import *
from algs import *

from evaluate.eval_funcs import *
from evaluate.evaluator import Evaluator
from visualize import SearchPlotter

plotter = SearchPlotter()

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/world_mug.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/world_mug_sim.xml", cam_config)

loss_func = StructuralSimilarity()

alg1 = ParticleSwarm(sim_viewer, loss_func=loss_func)
alg1_config = ParticleSwarm.Config(time_limit=100, population=30, num_iters=300)

alg2 = SimulatedAnnealing(sim_viewer, loss_func=loss_func)
alg2_config = SimulatedAnnealing.Config(time_limit=100, num_iters=100, samples_per_temp=150)

alg3 = RandomSampling(sim_viewer, loss_func=loss_func)
alg3_config = RandomSampling.Config(time_limit=100, num_samples=10000)

alg4 = DifferentialEvolution(sim_viewer, loss_func=loss_func)
alg4_config = DifferentialEvolution.Config(time_limit=100, population=30, mut_prob=0.3)

alg5 = UniformSampling(sim_viewer, loss_func=loss_func)
alg5_config = UniformSampling.Config(time_limit=100, min_samples=10000, randomized=False)

evaluator = Evaluator(world_viewer, eval_func=IOU_Diff(method="mae"))

init_location = (0, 1.3, 0.3)
random_orientations = np.random.uniform(0, 2 * np.pi, size=(5, 3))
init_positions = [ObjectPosition(orient, init_location) for orient in random_orientations]

alg5.register_callback(plotter.add_data)

for alg, config in [(alg5, alg5_config)]:
    losses = evaluator.evaluate(alg, config, init_positions)
    print(f"{type(alg).__name__}: {losses}")

In [ ]:
cv.destroyAllWindows()